# 양방향 LSTM과 CRF(Bidirectional LSTM + CRF)

## load training data

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('../../training_data/NERTagger/BIO_TAGGER_TRAINING_DATASET.csv')

In [2]:
data = data.fillna(method = "ffill") # Null값 
data.head(5)

,Unnamed: 0,Sentence #,Word,Pos,Tag
0,0,Sentence 0,what,WP,O
1,1,Sentence 0,is,VBZ,O
2,2,Sentence 0,the,DT,O
3,3,Sentence 0,cost,NN,O
4,4,Sentence 0,of,IN,O


In [3]:
# 필요없는 column을 제거
del data['Pos']
del data['Unnamed: 0']

In [4]:
# 단어의 소문자화
data['Word'] = data['Word'].str.lower()
data.head(5)

,Sentence #,Word,Tag
0,Sentence 0,what,O
1,Sentence 0,is,O
2,Sentence 0,the,O
3,Sentence 0,cost,O
4,Sentence 0,of,O


In [5]:
# 중복을 허용하지 않고, 단어들을 모아 단어 집합을 만듦
# 단어 dictionary 만들기 
vocab = (list(set(data["Word"].values)))
print(vocab)

['schedule', 'c', 'seven', 'wants', 'get', 'only', 'level', 'snack', 'take', 'miami', 'about', 'reverse', 'rates', 'indiana', 'back', 'dca', 'georgia', 'll', 'just', 'long', 'evening', 'way', 'mco', 'thank', 'listing', 'milwaukee', 'atl', 'carried', 'meals', 'connections', 'local', '730', 'scheduled', 'snacks', 'got', '1045', '7', 'connecting', 'goes', 'runs', 'town', '270', 'provide', 'including', 'year', 'afternoons', 'limousine', 'eleventh', 'return', 'repeating', 'us', 'la', 'another', 'operation', 'departures', 'time', 'round', 'like', 'sb', '1100', '1209', 'stopovers', 'near', 'weekday', 'first', 'cost', 'west', '1110', 'during', 'prices', '382', 'minnesota', 'minimum', 'listed', '755', 'anything', 'be1', 'area', 'via', 'landing', 'okay', 'departs', 'actually', 'ten', 'taxi', 'meal', 'right', '1201', '20', 'besides', 'than', 'next', '1800', 'hp', '727', 'midway', 'fn', 'start', 'nighttime', 'cincinnati', 'both', 'over', 'called', 'baltimore', 'travels', 'sunday', 'red', 'yx', 'so

In [6]:
# 중복을 허용하지 않고, 태그들을 모아 태그 집합을 만듦
# 태그 dictionary 만들기
tags = list(set(data["Tag"].values))
print(len(tags))

88


In [7]:
data.head(5)

,Sentence #,Word,Tag
0,Sentence 0,what,O
1,Sentence 0,is,O
2,Sentence 0,the,O
3,Sentence 0,cost,O
4,Sentence 0,of,O


In [8]:
# 하나의 문장에 등장한 단어와 개체명 태깅 정보끼리 pair로 묶음
# example : (what, O)

#temp의 "Word" 컬럼의 값들을 리스트로 ( temp["Word"].values.tolist() )
#tepm의 "Tag" 컬럼의 값들을 리스트로 ( temp["Tag"].values.tolist() ) 만들어 zip하고 
#각 리스트의 값들을 순서대로 하나씩 가져와 각각 w, t로 하여 pair (w,t)를 만든다 
func = lambda temp: [(w, t) for w, t in zip(temp["Word"].values.tolist(), temp["Tag"].values.tolist())]
#data의 'Sentence #'컬럼의 값별로 그룹화하여 func 적용한다
All_data=[t for t in data.groupby("Sentence #").apply(func)]

In [9]:
All_data[0]

[('what', 'O'),
 ('is', 'O'),
 ('the', 'O'),
 ('cost', 'O'),
 ('of', 'O'),
 ('a', 'O'),
 ('round', 'B-round_trip'),
 ('trip', 'I-round_trip'),
 ('flight', 'O'),
 ('from', 'O'),
 ('pittsburgh', 'B-fromloc'),
 ('to', 'O'),
 ('atlanta', 'B-toloc'),
 ('beginning', 'O'),
 ('on', 'O'),
 ('april', 'B-depart_date.month_name'),
 ('twenty', 'B-depart_date.day_number'),
 ('fifth', 'I-depart_date.day_number'),
 ('and', 'O'),
 ('returning', 'O'),
 ('on', 'O'),
 ('may', 'B-return_date.month_name'),
 ('sixth', 'B-return_date.day_number')]

In [10]:
# 단어와 개체명 태깅 정보를 분리하는 작업

import numpy as np

sentences, ner_tags = [], []

for tagged_sentence in All_data:
    sentence, ner_info = zip(*tagged_sentence) # 각 샘플에서 단어는 sentence에, 개체명 태깅 정보는 ner_infodp
    sentences.append(np.array(sentence)) # 각 샘플에서 단어 정보만 저장
    ner_tags.append(np.array(ner_info)) # 각 샘플에서 개체명 태깅 정보만 저장

In [11]:
import numpy as np
sentences, ner_tags = [], []
for tagged_sentence in All_data:
    sentence, ner_info = zip(*tagged_sentence) # 각 샘플에서 단어는 sentence에, 개체명 태깅 정보는 ner_infodp
    sentences.append(np.array(sentence)) # 각 샘플에서 단어 정보만 저장
    ner_tags.append(np.array(ner_info)) # 각 샘플에서 개체명 태깅 정보만 저장

In [12]:
# example
sentences[0]

array(['what', 'is', 'the', 'cost', 'of', 'a', 'round', 'trip', 'flight',
       'from', 'pittsburgh', 'to', 'atlanta', 'beginning', 'on', 'april',
       'twenty', 'fifth', 'and', 'returning', 'on', 'may', 'sixth'],
      dtype='<U10')

In [13]:
# example
ner_tags[0]

array(['O', 'O', 'O', 'O', 'O', 'O', 'B-round_trip', 'I-round_trip', 'O',
       'O', 'B-fromloc', 'O', 'B-toloc', 'O', 'O',
       'B-depart_date.month_name', 'B-depart_date.day_number',
       'I-depart_date.day_number', 'O', 'O', 'O',
       'B-return_date.month_name', 'B-return_date.day_number'],
      dtype='<U24')

In [14]:
# 단어 집합과 개체명 태깅 정보의 집합 만들기
# 단어 집합의 경우 Counter()를 이용해 단어의 등장 빈도수 계산

from collections import Counter

vocab = Counter()
tag_set = set()

for sentence in sentences: # 훈련 데이터 X에서 샘플을 1개씩 꺼내온다.
    for word in sentence: # 샘플에서 단어를 1개씩 꺼내온다.
        vocab[word.lower()] = vocab[word.lower()] + 1 # 각 단어의 빈도수 계산
        
for tags_list in ner_tags: # 훈련 데이터 y에서 샘플을 1개씩 꺼내온다.
    for tag in tags_list: # 샘플에서 개체명 정보를 1개씩 꺼내온다.
        tag_set.add(tag) # 각 개체명 정보에 대해서 중복을 허용하지 않고 집합을 만든다.

In [15]:
vocab

Counter({'what': 1387,
         'is': 698,
         'the': 2249,
         'cost': 71,
         'of': 412,
         'a': 1029,
         'round': 226,
         'trip': 241,
         'flight': 1317,
         'from': 4053,
         'pittsburgh': 651,
         'to': 4695,
         'atlanta': 730,
         'beginning': 1,
         'on': 1681,
         'april': 44,
         'twenty': 131,
         'fifth': 25,
         'and': 869,
         'returning': 13,
         'may': 53,
         'sixth': 24,
         'now': 36,
         'i': 1117,
         'need': 265,
         'leaving': 343,
         'fort': 94,
         'worth': 94,
         'arriving': 212,
         'in': 908,
         'denver': 968,
         'no': 9,
         'later': 8,
         'than': 42,
         '2': 23,
         'pm': 345,
         'next': 103,
         'monday': 174,
         'show': 1042,
         'me': 1260,
         'ground': 230,
         'transportation': 217,
         'houston': 111,
         'afternoon': 194,
        

In [16]:
len(tag_set)

88

In [17]:
# 단어 집합을 등장 빈도수를 기준으로 정렬
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)

In [18]:
vocab_sorted

[('to', 4695),
 ('from', 4053),
 ('flights', 2629),
 ('the', 2249),
 ('on', 1681),
 ('what', 1387),
 ('flight', 1317),
 ('me', 1260),
 ('i', 1117),
 ('san', 1045),
 ('show', 1042),
 ('boston', 1042),
 ('a', 1029),
 ('denver', 968),
 ('in', 908),
 ('and', 869),
 ('francisco', 860),
 ('atlanta', 730),
 ('is', 698),
 ('pittsburgh', 651),
 ('dallas', 648),
 ('all', 620),
 ('list', 602),
 ('baltimore', 569),
 ('philadelphia', 544),
 ('like', 535),
 ('are', 490),
 ('airlines', 464),
 ('of', 412),
 ('that', 405),
 ('between', 404),
 ('washington', 388),
 ('pm', 345),
 ('leaving', 343),
 ('please', 343),
 ('morning', 340),
 ('would', 311),
 ('fly', 297),
 ('city', 290),
 ('for', 290),
 ('wednesday', 267),
 ('first', 267),
 ('need', 265),
 ('fare', 265),
 ('after', 250),
 ('oakland', 247),
 ('trip', 241),
 ('there', 241),
 ('d', 237),
 ('ground', 230),
 ('cheapest', 227),
 ('round', 226),
 ('you', 223),
 ('transportation', 217),
 ('does', 216),
 ('which', 216),
 ('class', 215),
 ('arriving', 21

In [19]:
word_to_index = {'PAD' : 0, 'OOV' :1}
i = 1
# 인덱스 0은 각각 입력값들의 길이를 맞추기 위한 PAD(padding을 의미)라는 단어에 사용된다.
# 인덱스 1은 모르는 단어를 의미하는 OOV라는 단어에 사용된다.
for (word, frequency) in vocab_sorted :
    # if frequency > 1 :
    # 빈도수가 1인 단어를 제거하는 것도 가능하겠지만 이번에는 별도 수행하지 않고 진행함.
        i = i + 1
        word_to_index[word] = i

In [20]:
#빈도수로 나열된 단어별로 인덱스 부여
word_to_index

{'PAD': 0,
 'OOV': 1,
 'to': 2,
 'from': 3,
 'flights': 4,
 'the': 5,
 'on': 6,
 'what': 7,
 'flight': 8,
 'me': 9,
 'i': 10,
 'san': 11,
 'show': 12,
 'boston': 13,
 'a': 14,
 'denver': 15,
 'in': 16,
 'and': 17,
 'francisco': 18,
 'atlanta': 19,
 'is': 20,
 'pittsburgh': 21,
 'dallas': 22,
 'all': 23,
 'list': 24,
 'baltimore': 25,
 'philadelphia': 26,
 'like': 27,
 'are': 28,
 'airlines': 29,
 'of': 30,
 'that': 31,
 'between': 32,
 'washington': 33,
 'pm': 34,
 'leaving': 35,
 'please': 36,
 'morning': 37,
 'would': 38,
 'fly': 39,
 'city': 40,
 'for': 41,
 'wednesday': 42,
 'first': 43,
 'need': 44,
 'fare': 45,
 'after': 46,
 'oakland': 47,
 'trip': 48,
 'there': 49,
 'd': 50,
 'ground': 51,
 'cheapest': 52,
 'round': 53,
 'you': 54,
 'transportation': 55,
 'does': 56,
 'which': 57,
 'class': 58,
 'arriving': 59,
 'before': 60,
 'milwaukee': 61,
 'st.': 62,
 'with': 63,
 'afternoon': 64,
 'available': 65,
 'have': 66,
 'american': 67,
 'new': 68,
 'give': 69,
 'one': 70,
 'at': 7

In [21]:
# 태깅 정보에도 인덱스를 부여
tag_to_index = {'PAD' : 0}
i = 0
for tag in tag_set:
    i = i + 1
    tag_to_index[tag] = i

In [22]:
tag_to_index

{'PAD': 0,
 'B-state_name': 1,
 'B-fare_basis_code': 2,
 'B-arrive_time.time': 3,
 'B-aircraft_code': 4,
 'I-round_trip': 5,
 'B-city_name': 6,
 'I-depart_time.end_time': 7,
 'I-flight_time': 8,
 'B-arrive_time.time_relative': 9,
 'I-arrive_time.start_time': 10,
 'B-or': 11,
 'I-depart_time.time': 12,
 'I-arrive_date.day_number': 13,
 'B-toloc': 14,
 'B-day_name': 15,
 'B-airport_name': 16,
 'B-arrive_date.date_relative': 17,
 'B-stoploc': 18,
 'B-flight_days': 19,
 'B-arrive_time.start_time': 20,
 'B-airport_code': 21,
 'B-class_type': 22,
 'I-arrive_time.time': 23,
 'B-arrive_time.period_of_day': 24,
 'I-arrive_time.time_relative': 25,
 'B-round_trip': 26,
 'B-flight_time': 27,
 'I-class_type': 28,
 'I-fare_amount': 29,
 'B-flight_number': 30,
 'I-return_date.date_relative': 31,
 'O': 32,
 'B-return_date.day_number': 33,
 'I-cost_relative': 34,
 'B-meal': 35,
 'B-fromloc': 36,
 'B-arrive_date.month_name': 37,
 'B-depart_date.today_relative': 38,
 'B-depart_date.date_relative': 39,
 '

In [23]:
# 단어 정수 인코딩 진행
data_X = []

for s in sentences: # 전체 데이터에서 하나의 데이터. 즉, 하나의 문장씩 불러옵니다.
    temp_X = []
    for w in s: # 각 문장에서 각 단어를 불러옵니다.
        temp_X.append(word_to_index.get(w,1)) # 각 단어를 매핑되는 인덱스로 변환합니다.
    data_X.append(temp_X)

In [24]:
data_X[0]

[7,
 20,
 5,
 146,
 30,
 14,
 53,
 48,
 8,
 3,
 21,
 2,
 19,
 650,
 6,
 189,
 100,
 241,
 17,
 312,
 6,
 172,
 244]

In [25]:
# 개체명 태깅 정보 정수 인코딩 진행
data_y = []
for s in ner_tags:
    temp_y = []
    for w in s:
            temp_y.append(tag_to_index.get(w))
    data_y.append(temp_y)

In [26]:
data_y[0]

[32,
 32,
 32,
 32,
 32,
 32,
 26,
 5,
 32,
 32,
 36,
 32,
 14,
 32,
 32,
 65,
 62,
 85,
 32,
 32,
 32,
 56,
 33]

## set max_len to 45, padd data

In [27]:
max_len = 45
from keras.preprocessing.sequence import pad_sequences
pad_X = pad_sequences(data_X, padding = 'post', maxlen = max_len)
# data_X의 모든 샘플의 길이를 패딩할 때, 뒤의 공간을 숫자 0으로 채움
pad_y = pad_sequences(data_y, padding = 'post', value = tag_to_index['PAD'], maxlen = max_len)
# data_y의 모든 샘플의 길이를 패딩할 때, 'PAD'에 해당하는 인덱스로 채움
# 결과적으로 'PAD'의 인덱스 값인 0으로 패딩됨

Using TensorFlow backend.


In [28]:
pad_X[0]

array([  7,  20,   5, 146,  30,  14,  53,  48,   8,   3,  21,   2,  19,
       650,   6, 189, 100, 241,  17, 312,   6, 172, 244,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0])

In [29]:
pad_y[0]

array([32, 32, 32, 32, 32, 32, 26,  5, 32, 32, 36, 32, 14, 32, 32, 65, 62,
       85, 32, 32, 32, 56, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

## split data ( 0.8 for training, rest for testing )

In [30]:
# 훈련 데이터와 테스트 데이터를 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pad_X, pad_y, test_size = .1, random_state = 777)

In [31]:
# 단어 집합과 태깅 정보 집합의 크기를 변수에 저장
# 모델 생성에 사용할 변수
n_words = len(word_to_index)
n_labels = len(tag_to_index)

## make & train biLSTM-CRF model

In [32]:
# 모델에 양방향 LSTM을 사용, 모델의 출력층에 CRF층을 배치
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

model = Sequential()
model.add(Embedding(input_dim = n_words, output_dim = 20, input_length = max_len, mask_zero = True))
model.add(Bidirectional(LSTM(units = 50, return_sequences = True, recurrent_dropout = 0.1)))
model.add(TimeDistributed(Dense(50, activation = "relu")))
crf = CRF(n_labels)
model.add(crf)

W1214 00:03:38.141029  8724 deprecation_wrapper.py:119] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1214 00:03:38.154992  8724 deprecation_wrapper.py:119] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1214 00:03:38.156987  8724 deprecation_wrapper.py:119] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1214 00:03:38.260730  8724 deprecation_wrapper.py:119] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name 

In [33]:
from keras.utils import np_utils
y_train2 = np_utils.to_categorical(y_train) # one-hot 인코딩

In [34]:
pd.set_option('max_colwidth', 800)

In [35]:
y_train2[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [36]:
model.compile(optimizer = "rmsprop", loss = crf.loss_function, metrics = [crf.accuracy])
history = model.fit(X_train, y_train2, batch_size = 32, epochs = 7, validation_split = 0.1, verbose = 1)

c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras_contrib\layers\crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras_contrib\layers\crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '
W1214 00:03:38.971355  8724 deprecation_wrapper.py:119] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1214 00:03:40.947807  8724 deprecation_wrapper.py:119] From c:\users\yunja_kuj61s9\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:986

Train on 4399 samples, validate on 489 samples
Epoch 1/7
4399/4399 [==============================] - 27s 6ms/step - loss: 18.0963 - crf_viterbi_accuracy: 0.5996 - val_loss: 17.8478 - val_crf_viterbi_accuracy: 0.6540
Epoch 2/7
4399/4399 [==============================] - 25s 6ms/step - loss: 17.0947 - crf_viterbi_accuracy: 0.7564 - val_loss: 17.2276 - val_crf_viterbi_accuracy: 0.8194
Epoch 3/7
4399/4399 [==============================] - 31s 7ms/step - loss: 16.7601 - crf_viterbi_accuracy: 0.8393 - val_loss: 17.0514 - val_crf_viterbi_accuracy: 0.8517
Epoch 4/7
4399/4399 [==============================] - 36s 8ms/step - loss: 16.6214 - crf_viterbi_accuracy: 0.8772 - val_loss: 16.9308 - val_crf_viterbi_accuracy: 0.9017
Epoch 5/7
4399/4399 [==============================] - 34s 8ms/step - loss: 16.5145 - crf_viterbi_accuracy: 0.9104 - val_loss: 16.8419 - val_crf_viterbi_accuracy: 0.9288
Epoch 6/7
4399/4399 [==============================] - 33s 8ms/step - loss: 16.4350 - crf_viterbi_accur

In [37]:
y_test2 = np_utils.to_categorical(y_test)
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test2)[1]))

544/544 [==============================] - 2s 4ms/step

 테스트 정확도: 0.9549


In [38]:
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key

index_to_tag = {}
for key, value in tag_to_index.items():
    index_to_tag[value] = key

In [39]:
# 실제 모델에 대해 f1 score 구하기
def sequences_to_tag(sequences): # 예측값을 index_to_tag를 사용하여 태깅 정보로 변경하는 함수.
    result = []
    for sequence in sequences: # 전체 시퀀스로부터 시퀀스를 하나씩 꺼낸다.
        temp = []
        for pred in sequence: # 시퀀스로부터 예측값을 하나씩 꺼낸다.
            pred_index = np.argmax(pred) # 예를 들어 [0, 0, 1, 0 ,0]라면 1의 인덱스인 2를 리턴한다.
            temp.append(index_to_tag[pred_index].replace("PAD", "O")) # 'PAD'는 'O'로 변경
        result.append(temp)
    return result

In [40]:
from nltk import word_tokenize

In [49]:
test_sentence = "I'd like to go from new york to london departing next friday by delta airline"
test_sentence = "List all flights from incheon to jeju on december 5th."
test_sentence = "how about to china"
test_sentence = test_sentence.lower()
test_sentence = word_tokenize(test_sentence)

In [50]:
new_X = []
for w in test_sentence:
    try:
        new_X.append(word_to_index.get(w,1))
    except KeyError:
        new_X.append(word_to_index['OOV'])
      # 모델이 모르는 단어에 대해서는 'OOV'의 인덱스인 1로 인코딩

In [51]:
pad_new = pad_sequences([new_X], padding="post", value=0, maxlen=max_len)

In [52]:
p = model.predict(np.array([pad_new[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test_sentence, p[0]):
    print("{:15}: {:5}".format(w, index_to_tag[pred]))

Word           ||Prediction
how            : O    
about          : O    
to             : O    
china          : B-toloc


## save model

In [45]:
#모델 저장
model.save("_BIO_TAGGER.h5")

In [46]:
#dictionary 저장
import pickle
with open('_word_to_index.pickle','wb') as f:
    pickle.dump(word_to_index, f)

In [47]:
with open('_index_to_tag.pickle','wb') as f:
    pickle.dump(index_to_tag, f)

In [48]:
with open('X_test.pickle','wb') as f:
    pickle.dump(X_test, f)
with open('y_test.pickle','wb') as f:
    pickle.dump(y_test, f)